In [27]:
import chromadb
import numpy as np
import pandas as pd

# Vector DB - Chroma DB 구축

In [28]:
df = pd.read_csv('본회의처리안건_법률안.csv')

In [29]:
client = chromadb.PersistentClient(path="chromadb")

collection = client.get_or_create_collection(
    name="detail_law_table",
    metadata={"hnsw:space": "cosine"}
)

In [30]:
# 인덱스
ids = []
# 메타 데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장한다고 한다. 
documents = []

In [31]:
for idx in range(len(df)):
    item = df.iloc[idx]
    id = item['의안명'].replace(' ', '-')
    document = f"""(의안명: {item['의안명']}) (제안일: {item['제안일']}) (의결결과: {item['의결결과']}) - (총투표수:{item['총투표수']}, 찬성:{item['찬성']}, 반대:{item['반대']}, 기권:{item['기권']})
                 (위원회심사 의결일:{item['위원회심사_의결일']}, 본회의심의 의결일:{item['본회의심의_의결일']}, 정부이송일:{item['정부이송일']}, 공포일:{item['공포일']})
                 (링크URL:{item['링크URL']}"""
    meta = {
        "카테고리": item['카테고리']
    }
    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)

#  DB 저장

In [32]:
collection.add(
    documents = documents,
    metadatas=doc_meta,
    ids=ids
)

# DB 쿼리  

In [41]:
collection.query(
    query_texts=["자동차"],
    n_results=5,
    where={"카테고리":"교통"},
    where_document={"$contains": "자동차"}
)

{'ids': [['자동차관리법-일부개정법률안(대안)(국토교통위원장)']],
 'distances': [[0.6961982250213623]],
 'metadatas': [[{'카테고리': '교통'}]],
 'embeddings': None,
 'documents': [['(의안명: 자동차관리법 일부개정법률안(대안)(국토교통위원장)) (제안일: 2023-08-23) (의결결과: 원안가결) - (총투표수:210.0, 찬성:209.0, 반대:nan, 기권:1.0)\n                 (위원회심사 의결일:2023-04-20, 본회의심의 의결일:2023-08-24, 정부이송일:2023-09-01, 공포일:2023-09-14)\n                 (링크URL:https://likms.assembly.go.kr/bill/billDetail.do?billId=PRC_Y2T3W0W4G1C8U1E6P2U2E3W7K2F8D2']]}